In [1]:
using DynamicHMCModels

ProjDir = rel_path_d("..", "scripts", "12")

df = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(df) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
df[!, :logpop] = map((x) -> log(x), df[!, :population]);
df[!, :society] = 1:10;

first(df[!, [:total_tools, :logpop, :society]], 5)

struct m_12_06d_model{TY <: AbstractVector, TX <: AbstractMatrix,
  TS <: AbstractVector}
    "Observations (total_tools)."
    y::TY
    "Covariates (logpop)"
    X::TX
    "Society"
    S::TS
    "Number of observations (10)"
    N::Int
    "Number of societies (also 10)"
    N_societies::Int
end

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::m_12_06d_model)(θ)
    @unpack y, X, S, N, N_societies = problem   # extract the data
    @unpack β, α, σ = θ  # β : a, bp, α : a_society
    ll = 0.0
    ll += logpdf(Cauchy(0, 1), σ)
    ll += sum(logpdf.(Normal(0, σ), α)) # α[1:10]
    ll += logpdf.(Normal(0, 10), β[1]) # a
    ll += logpdf.(Normal(0, 1), β[2]) # a
    ll += sum(
      [loglikelihood(Poisson(exp(α[S[i]] + dot(X[i, :], β))), [y[i]]) for i in 1:N]
    )
    ll
end

Instantiate the model with data and inits.

In [4]:
N = size(df, 1)
N_societies = length(unique(df[!, :society]))
X = hcat(ones(Int64, N), df[!, :logpop]);
S = df[!, :society]
y = df[!, :total_tools]
p = m_12_06d_model(y, X, S, N, N_societies);
θ = (β = [1.0, 0.25], α = rand(Normal(0, 1), N_societies), σ = 0.2)
p(θ)

-182.19705519520178

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::m_12_06d_model) =
    as( (β = as(Array, size(p.X, 2)), α = as(Array, p.N_societies), σ = asℝ₊) )

problem_transformation (generic function with 1 method)

Wrap the problem with a transformation, then use Flux for the gradient.

In [6]:
P = TransformedLogDensity(problem_transformation(p), p)
#∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));
∇P = ADgradient(:ForwardDiff, P);

Tune and sample.

In [7]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0039 s/step ...done
MCMC, adapting ϵ (25 steps)
0.0042 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0034 s/step ...done
MCMC, adapting ϵ (100 steps)
0.0017 s/step ...done
MCMC, adapting ϵ (200 steps)
0.0012 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00066 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00062 s/step ...done
MCMC (1000 steps)
0.00048 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [8]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :α, :σ),Tuple{Array{Float64,1},Array{Float64,1},Float64}},1}:
 (β = [1.2203755612727307, 0.2378165738228704], α = [-0.1237179505928725, 0.11971607222613388, -0.09349239119345844, 0.4749656902667122, 0.06597443540569618, -0.3380957213705037, 0.27339373001931766, 0.012068134741907544, 0.29767652250978444, 0.14878470401886668], σ = 0.2540391369139499)        
 (β = [0.5538654425823543, 0.34991508881851197], α = [-0.49811724919833117, -0.10744907741730066, -0.07401932418409736, -0.020166009851455842, -0.13824806488420466, -0.6387847805565272, -0.19764554139418591, -0.7223177768917445, 0.05570455664277531, -0.8339835586651703], σ = 0.44935783126232953)
 (β = [2.0373566501323603, 0.1386912452201454], α = [-0.5012498284149148, 0.022919737763497094, -0.043114416310875064, 0.5849577882986725, 0.20299285868159506, -0.36385722211363236, 0.4266868281815913, 0.16012573053705398, 0.49487985984057575, 0.46953549477064416], σ = 0.6631257415913456)       
 (β = [-0.27996

Extract the parameter posterior means.

In [9]:
posterior_β = mean(posterior[i].β for i in 1:length(posterior))
posterior_α = mean(posterior[i].α for i in 1:length(posterior))
posterior_σ = mean(posterior[i].σ for i in 1:length(posterior))

0.3162411564068822

Effective sample sizes (of untransformed draws)

In [10]:
ess = mapslices(effective_sample_size, get_position_matrix(chain); dims = 1)
ess

1×13 Array{Float64,2}:
 865.901  860.172  1000.0  988.114  …  567.313  554.357  801.263  209.454

NUTS-specific statistics

In [11]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.93, min/25%/median/75%/max: 0.01 0.9 0.97 0.99 1.0
  termination: AdjacentTurn => 8% DoubledTurn => 92%
  depth: 2 => 0% 3 => 6% 4 => 93% 5 => 0%


CmdStan result

In [12]:
m_12_6_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                            Mean                SD               Naive SE             MCSE            ESS
            a          1.076167468  0.7704872560 0.01218247319 0.0210530022 1000.000000
           bp         0.263056273  0.0823415805 0.00130193470 0.0022645077 1000.000000
  a_society.1   -0.191723568  0.2421382537 0.00382854195 0.0060563054 1000.000000
  a_society.2    0.054569029  0.2278506876 0.00360263570 0.0051693148 1000.000000
  a_society.3   -0.035935050  0.1926364647 0.00304584994 0.0039948433 1000.000000
  a_society.4    0.334355037  0.1929971201 0.00305155241 0.0063871707  913.029080
  a_society.5    0.049747513  0.1801287716 0.00284808595 0.0043631095 1000.000000
  a_society.6   -0.311903245  0.2096126337 0.00331426674 0.0053000536 1000.000000
  a_society.7    0.148637507  0.1744680594 0.00275858223 0.0047660246 1000.000000
  a_society.8   -0.164567976  0.1821341074 0.00287979309 0.0034297298 1000.000000
  a_society.9    0.277066965  0.1758237250 0.00278001719 0.0055844175  991.286501
 a_society.10   -0.094149204  0.2846206232 0.00450024719 0.0080735022 1000.000000
sigma_society    0.310352849  0.1374834682 0.00217380450 0.0057325226  575.187461
";

Show means

In [13]:
[posterior_β, posterior_α, [posterior_σ]]

3-element Array{Array{Float64,1},1}:
 [1.1138301195940192, 0.25943353128574054]                                                                                                                                                                           
 [-0.1978887526076462, 0.03617271804735259, -0.044314618373848584, 0.3324531582922331, 0.04626053391099113, -0.3121525657507764, 0.1590760067945609, -0.16418496183383896, 0.27710295869553603, -0.08568801581464212]
 [0.3162411564068822]                                                                                                                                                                                                

End of m12.6d.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*